# TP2 - DB Normalization and Querying

The objectives of this TP are:
1. Apply normalization 1NF -> 2NF -> 3NF
2. Perform SQL queries on the normalized database

In this TP, we will use a database **`wine.db`** (available in the course's website) containing wine information related to 'production' and 'sales'. 

> Production <---> Wine <---> Sales


---

### Working with db files in Jupyter
- Python provides an interface for SQLite through the *sqlite3* module
- The **`%%sql`** magic builds upon it (and other tools) to enable the usage of SQL commands within a Jupyter Notebook as in common SQL clients.
- Before proceeding, make sure that **`wine.db`** is in the same path as this notebook.
  - If **`wine.db`** is not in the same path, an empty **`wine.db`** file will be created, resulting in errors in later steps of the TP.
- The SQLite module in Python commits transactions automatically, this means that any change in the DB is immediately written to the file, e.g. creating/deleting tables.
  -  For this reason, it is recommended to have a backup of **`wine.db`** as it is provided in the course's website.

---

**`wine.db`** contains the following unnormalized tables:

<center>**Master1**</center>

|*Attribute*|         *Description*          |
| -------   |--------------------------------|
| NV        | Wine number                    |
| CRU       | Vineyard or group of vineyards |
| DEGRE     | Alcohol content                |
| MILL      | Vintage year                   |
| QTE       | Number of bottles harvested    |
| NP        | Producer number                |
| NOM       | Producer's last name           |
| PRENOM    | Producer's first name          |
| REGION    | Production region              |

From wikipedia:

__Cru__: Often used to indicate a specifically named and legally defined vineyard or ensemble of vineyards and the vines "which grow on [such] a reputed terroir; by extension of good quality." The term is also used to refer to the wine produced from such vines.


<center>**Master2**</center>

|*Attribute*|                         *Description*                  |
| -------   |--------------------------------------------------------|
| NV        | Wine number                                            |
| CRU       | Vineyard or group of vineyards                         |
| DEGRE     | Alcohol content                                        |
| MILL      | Vintage year                                           |
| DATES     | Buying date                                            |
| LIEU      | Place where the wine was sold                          |
| QTE       | Number of bottles bought                               |
| NB        | Client (buveur) number                                 |
| NOM       | Client's last name                                     |
| PRENOM    | Client's first name                                    |
| TYPE      | Type of client by volume of purchases                  |
| REGION    | Administrative Region (different to production region) |


In [1]:
import sqlite3    # Python interface for SQLite databases

In [2]:
def printSchema(connection):
    # Function to print the DB schema
    # Source: http://stackoverflow.com/a/35092773/4765776
    for (tableName,) in connection.execute(
        """
        select NAME from SQLITE_MASTER where TYPE='table' order by NAME;
        """
    ):
        print("{}:".format(tableName))
        for (
            columnID, columnName, columnType,
            columnNotNull, columnDefault, columnPK,
        ) in connection.execute("pragma table_info('{}');".format(tableName)):
            print("  {id}: {name}({type}){null}{default}{pk}".format(
                id=columnID,
                name=columnName,
                type=columnType,
                null=" not null" if columnNotNull else "",
                default=" [{}]".format(columnDefault) if columnDefault else "",
                pk=" *{}".format(columnPK) if columnPK else "",
            ))

In [3]:
conn = sqlite3.connect('wine.db')
c = conn.cursor()
print("Database schema:")
printSchema(conn)           # An usefull way to viualize the content of the database

Database schema:
CLIENT:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(DATE)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)
PRODUCTION:
  0: NP(NUM)
  1: NV(NUM)
  2: QTE(NUM)
PRODUCTOR:
  0: NP(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: REGION(TEXT)
REGION_LIEU:
  0: LIEU(TEXT)
  1: REGION(TEXT)
SALES:
  0: NB(NUM)
  1: NV(NUM)
  2: QTE(NUM)
  3: LIEU(TEXT)
WINE:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
WINE2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)


From this point we will use __%%sql__ magic

In [4]:
%load_ext sql
%sql sqlite:///wine.db

u'Connected: None@wine.db'

# PART I: Database normalization

The first task on this TP is the normalization of the wine data. In its current state both tables **Master1** and **Master2** are in the First Normal Form (1NF).

By inspecting the content of these tables we can see that multiple tuples have NULL values.

In [5]:
%%sql SELECT *
FROM Master1
LIMIT 10;

Done.


NV,CRU,DEGRE,MILL,QTE,NP,NOM,PRENOM,REGION
None,None,None,None,None,3,Six,Paul,Alsace
None,None,None,None,None,6,Marmagne,Bernard,Bourgogne
None,None,None,None,None,8,Lioger d'Harduy,Gabriel,Bourgogne
None,None,None,None,None,16,Barbin,Bernard,Bourgogne
None,None,None,None,None,17,Faiveley,Guy,Bourgogne
None,None,None,None,None,18,Tramier,Jean,Bourgogne
None,None,None,None,None,19,Dupaquier,Roger,Bourgogne
None,None,None,None,None,20,Lamy,Jean,Bourgogne
None,None,None,None,None,21,Cornu,Edmond,Bourgogne
None,None,None,None,None,26,Violot,Gilbert,Bourgogne


* Notice that Jupyter *displays* 'None' instead of 'NULL'. 
  - This is only to comply with python notation.
* To account for NULL values, your SQL queries must test explicitly for 'NULL'.

Another problem in **Master1** and **Master2** is data redundancy, for example:

In [6]:
%%sql SELECT *
FROM Master1
WHERE NV = 45;

Done.


NV,CRU,DEGRE,MILL,QTE,NP,NOM,PRENOM,REGION
45,Chiroubles,None,1983,90,2,Boxler,Albert,Alsace
45,Chiroubles,None,1983,912,67,Descombes,Jean Ernest,Beaujolais
45,Chiroubles,None,1983,98,71,Chalandard,Danile,Jura
45,Chiroubles,None,1983,540,78,Michlel,Pierre Emile,Jura
45,Chiroubles,None,1983,450,86,Dumazet,Marc,Rhone


---

Additional resource for Normalization:

https://www.youtube.com/watch?v=UrYLYV7WSHM

---

#### Exercise 1.1

Convert table **Master1** to the Second Normal Form (2NF) or Third Normal Form (3NF) as applicable.
* Explain your answer
* List main functional dependencies (not all of them)
* Describe the schema of new tables and how they relate
  * You can write Tables as above or you can insert images in the notebook.
  
Remember that **`wine.db`** contains information related to wine 'production' and 'sells'.

> Production <---> Wine <---> Sales

A good start point is to look for the 'Wine' attributes.

**Hint:** Look for redundant information between the master tables.

# SOLUTION 
 
After exploring the data in details with some queries , we can easyly identify the following main functional dependencies:
- (NP,NV) ---> *   (**This is the primary key)**
- NP ---> (NOM, PRENOM, REGION) **This is a partial fd that shoud be removed**
- NV ---> ( CRU , DEGRE, MILL)  **This is a partial fd that should be removed** 
Since the tables are already in 1NF form, in oder to trasform **Master1** we have to ensure that: there are not partial dependencies in table. 
The primary key ( unique and minimal) is a **composite** primary key **[NV, NP]**. 
Having said that. It's clear and  intuitive that all informations about productor (Nom ,Prénom, Region)  depend only on the **NP** and the remaining informations (CRU, DEGRE, MILL ) depends on the wine number NV. So , we can split the the table in oder to avoid these partial dependencies. 
 ## First table: Productor
 ![alt text](./PRODUCTOR.png)
 
 ## Second table: Wine
 ![alt text](./WINE.png)
 
 ## Third table: Production
 ![alt text](./PRODUCTION.png) 
 
 




#### Exercise 1.2

Convert table **Master2** to the Second Normal Form (2NF) or Third Normal Form (3NF) as applicable.
* Explain your answer
* List main functional dependencies (not all of them)
* Describe the schema of new tables and how they relate
  * You can write Tables as above or you can insert images in the notebook.

**Note:** For this part, consider that a wine can be bought in multiple locations and multiple times per day.

In [7]:
%%sql SELECT NB ,NOM, PRENOM, TYPE
FROM Master2 
WHERE NV  is not NULL AND NB IS NOT NULL 
ORDER BY NB
LIMIT 5


Done.


NB,NOM,PRENOM,TYPE
1,Aristote,None,petit
2,Artaud,Antonin,moyen
2,Artaud,Antonin,moyen
2,Artaud,Antonin,moyen
3,Aron,Raymond,gros


# SOLUTION 
By proceding as we did before with the precedent table, we can identify the following functional dependencies: 
- (NV, NB) ---> all others    **This is the primary key** 
- NV ---> CRU, DEGRE, MILL     **This is a partial functional dependency**
- NB ---> NOM, PRENOM, TYPE    **This is a partial functional dependency**  
- LIEU ---> REGION      **this is a transitional fd ** 

Eliminating partial dependencies and transitives dependencies leads to this possible configuration of new tables: 
 ## First table: WINE2
 ![alt text](./WINE2.png) 
 ## Second table: CLIENT
 ![alt text](./CLIENT.png) 
 ## Third table: SALES
 ![alt text](./SALES.png)  
 ## Third table: REGION_LIEU 
 With the hope that a LOCATION is uniquely identified by it's name as it's actually the case in the DB.
 ![alt text](./REGION_LIEU.png) 
 




Once you have defined the 2NF or 3NF (as applicable) we need to split the data into new tables.

A table can be created from the result of a query.

In the following example we will create a new table "dummy" to store the different values of alcohol content.

In [8]:
%%sql DROP TABLE IF EXISTS dummy;

-- Create dummy table
CREATE TABLE dummy AS
SELECT DISTINCT DEGRE
FROM MASTER1;

Done.
Done.


[]

In [9]:
print("\nContent of the database")
printSchema(conn)


Content of the database
CLIENT:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(DATE)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)
PRODUCTION:
  0: NP(NUM)
  1: NV(NUM)
  2: QTE(NUM)
PRODUCTOR:
  0: NP(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: REGION(TEXT)
REGION_LIEU:
  0: LIEU(TEXT)
  1: REGION(TEXT)
SALES:
  0: NB(NUM)
  1: NV(NUM)
  2: QTE(NUM)
  3: LIEU(TEXT)
WINE:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
WINE2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
dummy:
  0: DEGRE(NUM)


In [10]:
%%sql
SELECT *
FROM dummy;

Done.


DEGRE
None
11.5
11.3
12.1
10.9
11.7
11.2
12.3
11.9
11.8


Looking into "dummy", we notice that our query includes NULL. This is not allowed if we were to use DEGRE as key for a table.

To correct this, we need to change the query to explicitly test for NULL as follows:

In [11]:
%%sql DROP TABLE IF EXISTS dummy;

-- Create dummy table
CREATE TABLE dummy AS
SELECT DISTINCT DEGRE
FROM MASTER1
WHERE DEGRE IS NOT NULL;

SELECT *
FROM dummy;

Done.
Done.
Done.


DEGRE
11.5
11.3
12.1
10.9
11.7
11.2
12.3
11.9
11.8
10.7


Notice that we use `NULL` given that `None` is only used for display.

In [12]:
# Remove "dummy" table
%sql DROP TABLE IF EXISTS dummy;

Done.


[]

In [13]:
%%sql 
select distinct CRU
from master1  
where cru='Pommard'
order by cru 
limit 15


Done.


CRU
Pommard


#### Exercise 1.3

Create the new tables from Master1:

In [14]:
%%sql 
-- Create PRODUCTOR TABLE 
DROP TABLE IF EXISTS PRODUCTOR;
CREATE TABLE PRODUCTOR AS
SELECT DISTINCT NP, NOM, PRENOM, REGION
FROM MASTER1
WHERE NP IS NOT NULL; 

-- Create WINE TABLE 
DROP TABLE IF EXISTS WINE;
CREATE TABLE WINE AS
SELECT  DISTINCT NV, CRU, DEGRE, MILL
FROM MASTER1
WHERE NV IS NOT NULL; 

--  Create PRODUCTION TABLE 
DROP TABLE IF EXISTS PRODUCTION;
CREATE TABLE PRODUCTION AS
SELECT   NP,NV, QTE
FROM MASTER1
WHERE NV IS NOT NULL or NP is not null; 
-- Create PRODUCTION TABLE 
DROP TABLE IF EXISTS PRODUCTION;
CREATE TABLE PRODUCTION AS
SELECT   NP,NV, QTE
FROM MASTER1
WHERE NV IS NOT NULL or NP is not null; 



Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

#### Exercise 1.4

Create the new tables from Master2:

In [15]:
%%sql 

-- Create WINE2 TABLE 
DROP TABLE IF EXISTS WINE2;
CREATE TABLE WINE2 AS
SELECT  DISTINCT NV, CRU, DEGRE, MILL
FROM MASTER2
WHERE NV IS NOT NULL; 
-- Create CLIENT TABLE 
DROP TABLE IF EXISTS CLIENT;
CREATE TABLE CLIENT AS
SELECT  DISTINCT NB, NOM, PRENOM,TYPE
FROM MASTER2
WHERE NB IS NOT NULL; 
--- Create SALES TABLE 
DROP TABLE IF EXISTS SALES;
CREATE TABLE SALES AS
SELECT  NB,NV, QTE, LIEU
FROM MASTER2
WHERE NV IS NOT NULL or NB is not null; 
--- Create REGION_LIEU TABLE 
DROP TABLE IF EXISTS REGION_LIEU;
CREATE TABLE REGION_LIEU AS
SELECT  DISTINCT LIEU, REGION
FROM MASTER2
WHERE NV IS NOT NULL or NB is not null; 







Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

# PART II: SQL QUERIES

In the second part of this TP you will create SQL queries to retrieve information from the database.

**Important:**

- You MUST use the normalized tables created in previous steps.
  - The normalized tables will also be used in TP3.
- Do NOT use **Master1** and **Master2** in your queries.

#### Exercise 2.1

What are the different types of clients (buveurs) by volume of purchases?

In [16]:
%%sql 
select distinct type 
from CLIENT 
limit 10

Done.


TYPE
petit
moyen
gros


#### Exercise 2.2

What regions produce Pommard or Brouilly?

In [17]:
%%sql
select distinct REGION
from PRODUCTION Prod, PRODUCTOR P, WINE W 
where cru= "Pommard" or cru="Brouilly"     
       and  Prod.NP=P.NP
       and  Prod.NV=W.NV 
LIMIT 12
       



Done.


REGION
Alsace
Bourgogne
Beaujolais
Jura
Rhone
Provence
Corse
Languedoc
Pyrenees
Sud


#### Exercise 2.3

What regions produce Pommard and Brouilly?

In [18]:
%%sql
select distinct REGION
from PRODUCTION Prod, PRODUCTOR P, WINE W 
where cru= "Pommard"     
       and  Prod.NP=P.NP
       and  Prod.NV=W.NV
        and REGION IN ( SELECT REGION 
                   from PRODUCTION Prod, PRODUCTOR P, WINE W 
                   where cru="Brouilly"     
                   and  Prod.NP=P.NP
                   and  Prod.NV=W.NV )


Done.


REGION
Bourgogne


#### Exercise 2.4

Get the number of wines bught by CRU and Millésime

In [19]:
%%sql
select CRU, MILL , count(*) as Result
from  SALES S, WINE2 W2
where S.NV= W2.NV 
GROUP BY CRU, MILL
ORDER BY Result desc


Done.


CRU,MILL,Result
Mercurey,1980,6
Richebourg,1943,5
Beaujolais Villages,1979,3
Chapelle Chambertin,1973,3
Cotes de Beaune Villages,1975,3
Savigny les Beaunes,1978,3
Seyssel,1974,3
Beaujolais Primeur,1983,2
Chiroubles,1983,2
Clairette de Die,1976,2


#### Exercise 2.5

Retrieve the wine number (NV) of wines produced by more than three producers

In [20]:
%%sql
select  NV
from PRODUCTION
GROUP BY NV 
HAVING count(DISTINCT NP)>3
limit 5



Done.


NV
None
45
78
89
98


#### Exercise 2.6

Which producers have not produced any wine?

In [21]:
%%sql 
select P.np
from PRODUCTION Prod , PRODUCTOR P
where Prod.np=P.np 
GROUP BY p.np
HAVING count(DISTINCT NV)=0
LIMIT 5

Done.


NP
3
6
8
16
17


#### Exercise 2.7

What clients (buveurs) have bought at least one wine from 1980?

In [22]:
%%sql 
select S.NB 
from SALES S , WINE2 W2 
where S.NV=W2.NV 
      and W2.MILL= 1980 
GROUP BY NB 
HAVING count(*)>=1


Done.


NB
2
8
44
45
50
61


#### Exercise 2.8

What clients (buveurs) have NOT bought any wine from 1980?

In [23]:
%%sql 
select *
from  CLIENT
where NB not in (select S.NB 
               from SALES S , WINE2 W2 
               where S.NV=W2.NV 
               and W2.MILL= 1980 
               GROUP BY NB 
               HAVING count(*)>=1
              )
      

Done.


NB,NOM,PRENOM,TYPE
11,Breton,Andre,petit
13,Barthes,Roland,moyen
16,Balzac,Honore de,moyen
18,Celine,Louis Ferdinand,gros
20,Chateaubriand,Francois-Rene de,moyen
21,Corbiere,Tristan,petit
23,Corneille,Pierre,petit
25,Char,Rene,petit
27,Dumas,Alexandre,gros
29,Fournier,Alain,petit


#### Exercise 2.9

What clients (buveurs) have bought ONLY wines from 1980?

In [83]:
%%sql 
select distinct NB 
 from SALES S , WINE2 W2 
 where S.NV=W2.NV 
       and W2.MILL=1980
       and NB  not in (select distinct NB
                      from SALES S , WINE2 W2 
                       where S.NV=W2.NV 
                       and W2.MILL != 1980  )
              

Done.


NB


Curiously this query gives a null result even if the outer query and the inner query give the same results as int the following query which gives the correct result. 

In [82]:
%%sql 
select distinct nb 
from master2 
where nb is not null 
         and mill=1980 
       and nb not in (select distinct nb 
from master2 
where nb is not null 
         and mill!=1980 
    order by nb )

Done.


NB
44
45
50


#### Exercise 2.10

List all wines from 1980

In [25]:
%%sql
select * 
from WINE2 
WHERE MILL=1980

Done.


NV,CRU,DEGRE,MILL
1,Mercurey,11.5,1980
4,Mercurey,10.9,1980
16,Meursault,12.1,1980
20,Cote de Brouilly,12.1,1980
26,Chateau Corton Grancey,None,1980
28,Volnay,11,1980
43,Fleurie,11.4,1980
74,Arbois,12,1980
78,Etoile,12,1980
79,Seyssel,11,1980


#### Exercise 2.11

What are the wines from 1980 bought by NB=2?

In [26]:
%%sql
select S.NV, W2.CRU, W2.DEGRE, W2.MILL
from SALES S, WINE2 W2 
WHERE S.nv=W2.nv 
      and S.nb=2 
      and W2.mill=1980 
      

Done.


NV,CRU,DEGRE,MILL
1,Mercurey,11.5,1980


#### Exercise 2.12

What clients (buveurs) have bought ALL the wines from 1980?

In [27]:
%%sql 
select nb 
from master2 
where nb is not null and mill=1980 
group by nb 
having count(*)=(select count(distinct nv)
                  from master2
                   where mill=1980)

Done.


NB
44
